### Environment requirement

* Standard Library: 
    * [io](https://docs.python.org/2/library/io.html)
    * [time](https://docs.python.org/2/library/time.html)
    * [json](https://docs.python.org/2/library/json.html)

* Third Party
    * [requests](http://docs.python-requests.org/en/master/)
    * [spotipy](https://pypi.org/project/spotipy/): pip3 install spotipy

In [25]:
import spotipy
import spotipy.oauth2 as oauth2
import io, time, json
import requests
from pathlib import Path
import csv
import pandas as pd
from fastai.collab import *

In [2]:
def read_client_info(filepath):
    """
    Read the Spotify Client ID from file.
    
    Args:
        filepath (string): File containing API Key
    Returns:
        client_id (string): The client_id
    """
    return Path(filepath).read_text().strip()

def generate_token():
    """ Generate the token from client_id and client_secret """
    credentials = oauth2.SpotifyClientCredentials(
        client_id=read_client_info("client_id.txt"),
        client_secret=read_client_info("client_secret.txt"))
    token = credentials.get_access_token()
    return token


In [3]:
# def write_tracks(text_file, tracks):
#     with open(text_file, 'a') as file_out:
#         while True:
#             for item in tracks['items']:
#                 if 'track' in item:
#                     track = item['track']
#                 else:
#                     track = item
#                 try:
#                     track_url = track['external_urls']['spotify']
#                     file_out.write(track_url + '\n')
#                 except KeyError:
#                     print(u'Skipping track {0} by {1} (local only?)'.format(
#                             track['name'], track['artists'][0]['name']))
#             # 1 page = 50 results
#             # check if there are more pages
#             if tracks['next']:
#                 tracks = spotify.next(tracks)
#             else:
#                 break

# def write_playlist(username, playlist_id):
#     results = spotify.user_playlist(username, playlist_id,
#                                     fields='tracks,next,name')
#     text_file = u'{0}.txt'.format(results['name'], ok='-_()[]{}')
#     print(u'Writing {0} tracks to {1}'.format(
#             results['tracks']['total'], text_file))
#     tracks = results['tracks']
#     write_tracks(text_file, tracks)

In [18]:
# def get_list_of_category(token):
#     """
#     Make an authenticated request to the Spotify API to get a list of browse categories.
#     """
#     headers = {'Authorization': 'Bearer %s' % token}
#     params = {"limit" : 50, "offset" : 0}
#     res = requests.get("https://api.spotify.com/v1/browse/categories", headers=headers,params=params)
# #     print(res.text)
#     d = json.loads(res.text)
#     d = d['categories']
# #     print(d['total'])
# #     print(len(d['items']))
#     return d

# def get_all_categories_id(items):
#     res = []
#     for item in items:
#         res.append(item['href'])
#     return res
    
# def get_all_playlists_from_category(token, end_point):
#     """
#     Make an authenticated request to the Spotify API to get a list of playlists from the category.

#     """
#     import pdb
    
#     headers = {'Authorization': 'Bearer %s' % token}
# #     params = {"limit" : 50, "offset" : 0}
#     end_point = end_point+"/playlists/?offset=0&limit=50"
#     ret = []
#     while end_point:
# #         pdb.set_trace()
#         res = requests.get(end_point, headers=headers)
#         print(res.status_code)
#         if res.status_code == 200:
#             d = json.loads(res.text)
#             d = d['playlists']
#             items = d['items']
#             end_point = d['next']
#             for item in items:
#                 tmp_d = {}
#                 tmp_d['name'] = item['name']
#                 tmp_d['href'] = item['href']
#                 tmp_d['id'] = item['id']
#                 ret.append(tmp_d)
    
#     print(len(ret))
#     return ret

def get_response(token, uri):
    headers = {'Authorization': 'Bearer %s' % token}
    rsp = requests.get(uri, headers=headers)
    return rsp.status_code, rsp.text

def get_tracks_from_playlist(token, playlist_uri):
    """
    Make an authenticated request to the Spotify API to get all the tracks in a public playlist.
    """
#     import pdb
    res = []
    status_code, txt = get_response(token, playlist_uri)
#     pdb.set_trace()
    if status_code == 200:
        d = json.loads(txt)
        tracks = d['tracks']
        while True:
            for item in tracks['items']:
                if item['track']:
                    track_name = item['track']['name']
                    res.append(track_name)
            if tracks['next']:
                tracks = spotify.next(tracks)
            else:
                break
    return res

def get_users_public_playlist(token, user_id):
    """
    Make an authenticated request to the Spotify API to get users public playlist.
    """
    res = []
    nxt = "https://api.spotify.com/v1/users/" + user_id + "/playlists/?limit=50&offset=0"
    while nxt:
        status, txt = get_response(token, nxt)
        if status == 200:
            d = json.loads(txt)
            nxt = d['next']
            items = d['items']
            for item in items:
                tmp_d = {}
                tmp_d['playlist_id'] = item['id']
                tmp_d['playlist_name'] = item['name']
                track_names = get_tracks_from_playlist(token,item['href'])
                tmp_d['track_names'] = track_names
                res.append(tmp_d)
        else:
            return None
    return res

In [20]:
token = generate_token()
spotify = spotipy.Spotify(auth=token)

# example playlist
playlists = get_users_public_playlist(token,"%23obviously")
with open('playlists.json', 'w') as fp:
    json.dump(playlists, fp)

1

In [37]:
matrix = pd.read_csv("small.csv")
ratings = [1] * matrix.shape[0]
matrix['ratings'] = ratings
matrix = matrix['trackname']
matrix.head()

KeyError: 'trackname'